<a href="https://colab.research.google.com/github/xenx96/OASIS_Ozone_Sensor_lstm/blob/main/OASIS_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

my_path = '/content/notebooks'
# Colab Notebooks 안에 my_env 폴더에 패키지 저장
#os.symlink('/content/drive/My Drive/Colab Notebooks/my_env', my_path)
sys.path.insert(0, my_path)

#VERSION = "20200220"
#!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
#!python pytorch-xla-env-setup.py 


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
Random_Seed = 100
torch.manual_seed(Random_Seed)

import torchvision
print(torch.cuda.get_device_name(0))
device = torch.device('cuda:0')


import warnings
warnings.filterwarnings('ignore')







data_path='/content/drive/MyDrive/MK.Corp/ModelingData.csv'
data = pd.read_csv(data_path)


Tesla T4


In [ ]:
#이건 데이터마다 다른 특성 및 데이터 핸들링 반영
def personal_data_set(data):
  data.x = data.x
  data.y = data.y
  data.z = data.z
  data.ppm[data.ppm>=2] = 2
  print(data.head())
  print(data.shape)
  return data

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler = MinMaxScaler()

#데이터의 주기를 주기
def create_sequences(array, seq_length):
    res = []
    if seq_length == 1 :
        for i in range(len(array)):
            tmp=array[i:(i+seq_length)]
            res.append(tmp)
    else : 
        for i in range(len(array)-seq_length-1):
            tmp = array[i:(i+seq_length)]
            res.append(tmp)
    return res

#스케일링 작업 
#GPU를활용한 텐서 작업.
def data_set(data,cols,target,seq_length,GPU=True):
  '''
  cols =[] #X :: X Values
  traget ='' #Y :: Traget Value
  seq_length = 2 :: 시퀀스 길이
  GPU = Using GPU :: GPU
  '''
  if GPU:
    device = torch.device('cuda:0')
  else :
    device = torch.device('CPU')
  
  X = data[cols]
  Y = data[target]
  #Create Scaler 
  xScaler = scaler.fit(X)
  yScaler = scaler.fit(Y.values.reshape(-1,1))
  #Transform Scaler
  X = xScaler.transform(X)
  Y = yScaler.transform(Y.values.reshape(-1,1))
  Y = Y.flatten()

  #Print and check the Shape
  print('X Data Set : ' , X.shape)
  print('Y Data Set : ' , Y.shape)
  
  X = create_sequences(X,seq_length)
  Y = create_sequences(Y,seq_length)

  X = torch.tensor(X).float().to(device)
  Y = torch.tensor(Y).float().to(device)

  #After Tensor Use with GPU and PRinting
  print('X Tensor Size : ',X.shape)
  print('Y Tensor Size : ',Y.shape)

  return X, Y



In [ ]:
data = personal_data_set(data)
print(f'Colnames of Data : {list(data.columns)}')

   Unnamed: 0    ppm    x   y  z  t  ...  right  left  up  s_name       svm  test_svm
0           1  0.096  110  65  0  1  ...      0     0   0      s1  0.200000 -0.104000
1           2  0.172  110  65  0  2  ...      0     0   0      s1  0.200000 -0.028000
2           3  0.285  110  65  0  3  ...      0     0   0      s1  0.200000  0.085000
3           4  0.432  110  65  0  4  ...      0     0   0      s1  0.269216  0.162784
4           5  0.609  110  65  0  5  ...      0     0   0      s1  0.426623  0.182377

[5 rows x 15 columns]
(2848, 15)
Colnames of Data : ['Unnamed: 0', 'ppm', 'x', 'y', 'z', 't', 's16', 'forward', 'back', 'right', 'left', 'up', 's_name', 'svm', 'test_svm']


In [ ]:
cols =['x', 'y', 'z', 't', 's16', 'forward', 'back', 'right', 'left', 'up']
target = 'ppm'
seq_length = 2
X_train, Y_train = data_set(data,cols,target,seq_length)

In [ ]:
class OASIS_LSTM(nn.Module):
    
    def __init__(self, n_features, n_hidden, seq_len, n_layers=2):
        super(OASIS_LSTM,self).__init__()
        self.n_hidden = n_hidden
        self.seq_len = seq_len
        self.n_layers = n_layers
        
        self.lstm = nn.LSTM(
        input_size = n_features,
        hidden_size = n_hidden,
        num_layers = n_layers,
        #dropout = 1
        ).to(device)
        
        self.linear = nn.Linear(in_features = n_hidden, out_features = 1).to(device)
        
    def reset_hidden_state(self):
        
        self.hidden = (
            torch.zeros(self.n_layers,self.seq_len,self.n_hidden).to(device),
            torch.zeros(self.n_layers,  self.seq_len,self.n_hidden).to(device))
        
    def forward(self,sequences):
        
        lstm_out, self.hidden = self.lstm(sequences.view(len(sequences),self.seq_len, -1), self.hidden)
        last_time_step = lstm_out.view(self.seq_len,len(sequences),self.n_hidden)[-1]
        y_pred = self.linear(last_time_step).to(device)
        
        return y_pred
        

In [ ]:
from tqdm.auto import tqdm
def train_model(model, train_data, train_labels,test_data = None, test_labels = None, num_epochs=250, lr=1e-3):
    loss_fn = torch.nn.MSELoss().to(device)
    
    optimiser = torch.optim.Adam(model.parameters(), lr=lr, weight_decay = 1e-4)
    num_epochs = num_epochs
    
    train_hist = np.zeros(num_epochs)
    test_hist = np.zeros(num_epochs)
    
    for t in tqdm(range(num_epochs), desc="OASIS LSTM is Learning", mininterval=0.01):
        model.reset_hidden_state()
        y_pred = model(X_train)
        loss = loss_fn(y_pred.float(),Y_train)
        
        if test_data is not None :
            with torch.no_grad():
                y_test_pred = model(X_test)
                test_loss = loss_fn(y_test_pred.float(), y_test)
            test_hist[t] = test_loss.item()
            
            if t % 10 == 0 :
                print(f'Epoch {t} train loss: {round(loss.item(),4)} test loss : {round(test_loss.item(),4)}' )
                
        elif t% 100 == 0 :
          print(f'Epoch {t} train loss : {loss.item()}')
            
        train_hist[t] = loss.item()
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
            
    return model.eval(), train_hist, test_hist
            
        

In [ ]:
# Hyper - Parameter Option Setting
n_features = X_train.shape[-1]
n_hidden = 64
n_layers = 2
lr = 1e-4
num_epochs = 1000

In [ ]:

model = OASIS_LSTM(n_features=n_features, n_hidden = n_hidden, seq_len = seq_length, n_layers= n_layers)
model, train_hist, test_hist = train_model(model, X_train,Y_train,num_epochs=num_epochs,lr = lr)

Epoch 0 train loss : 0.7764981389045715
Epoch 100 train loss : 0.161820650100708
Epoch 200 train loss : 0.1586267650127411
Epoch 300 train loss : 0.15859173238277435
Epoch 400 train loss : 0.15849633514881134
Epoch 500 train loss : 0.1583954095840454
Epoch 600 train loss : 0.15837064385414124
Epoch 700 train loss : 0.15834788978099823
Epoch 800 train loss : 0.15833055973052979
Epoch 900 train loss : 0.15832465887069702



In [ ]:
torch.save(model,'LSTM_OASIS.pt')
torch.save(model.state_dict(),'model_state_dict.pt')

In [ ]:
with torch.no_grad():
  test_seq = X_train
  preds =[]
  for i in range(X_train.shape[0]):
    y_test_pred = model(test_seq[i:i+1])
    pred = torch.flatten(y_test_pred).item()
    preds.append(pred)

In [ ]:
# Hyper - Parameter Option Setting
if(i ==4444):
  lr = 1e-4
  num_epochs = 1000
  n_features = X_train.shape[-1]
  for n_hidden in tqdm(range(64,179)):
    for n_layers in range(2,11):
      
      print(f'hidden : {n_hidden}, n_layer :{n_layers}')
      model = OASIS_LSTM(n_features=n_features, n_hidden = n_hidden, seq_len = seq_length, n_layers= n_layers)
      model, train_hist, test_hist = train_model(model, X_train,Y_train,num_epochs=num_epochs,lr = lr)
      torch.save(model,'LSTM_OASIS.pt')
      torch.save(model.state_dict(),'model_state_dict.pt')
    
      
      with torch.no_grad():
        test_seq = X_train
        preds =[]
        for i in range(X_train.shape[0]):
            y_test_pred = model(test_seq[i:i+1])
            pred = torch.flatten(y_test_pred).item()
            preds.append(pred)
      pred_diff = yScaler.inverse_transform(np.array(Y_train.cpu()))[:,[-1]] -yScaler.inverse_transform(np.array(preds).reshape(-1,1))
      if(max(abs(pred_diff))<=0.3):
        print(max(pred_diff))
        break;
      print(max(pred_diff))

In [ ]:
max(preds)

0.8825141191482544

In [ ]:
pred_value = yScaler.inverse_transform(np.array(preds).reshape(-1,1))


In [ ]:
pred_value

array([[1.1638733 ],
       [1.18034272],
       [1.18212962],
       ...,
       [1.23176458],
       [1.22773724],
       [1.22438744]])

In [ ]:
Result=yScaler.inverse_transform(np.array(Y_train.cpu()))[:,[-1]]-pred_value
Result=pd.DataFrame(Result)

In [ ]:
pd.concat([data,pd.DataFrame(pred_value)],axis=1)


,ppm,x,y,z,t,s16,forward,back,right,left,up,svm,test_svm,0
0,0.096,110,65,0,1,0.096,1,0,0,0,0,0.200000,-0.104000,1.163873
1,0.172,110,65,0,2,0.172,1,0,0,0,0,0.200000,-0.028000,1.180343
2,0.285,110,65,0,3,0.285,1,0,0,0,0,0.200000,0.085000,1.182130
3,0.432,110,65,0,4,0.432,1,0,0,0,0,0.269216,0.162784,1.170160
4,0.609,110,65,0,5,0.609,1,0,0,0,0,0.426623,0.182377,1.143529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2843,2.000,280,130,0,174,2.923,1,0,1,0,0,2.199428,0.135572,NaN
2844,2.000,280,130,0,175,2.916,1,0,1,0,0,2.184870,0.145130,NaN
2845,2.000,280,130,0,176,2.909,1,0,1,0,0,2.170538,0.155462,NaN
2846,2.000,280,130,0,177,2.901,1,0,1,0,0,2.156269,0.168731,NaN


In [ ]:
Result.to_csv('Result_LSTM.csv')